In [ ]:
import polars as pl, pandas as pd, numpy as np, glob,os,json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
# from fastFM import sgd 
import lightgbm as lgb
import category_encoders as ce
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display
import pickle
from sklearn.metrics import roc_auc_score, mean_squared_error

In [68]:
paths = glob.glob('D:/allcode/hacks/Devcraft25/final_data_with_all_preprocessing/*.parquet')
json_file = "D:/allcode/hacks/Devcraft25/submission_final/bidder/saved/creative_userid_count.json"
with open(json_file, "r") as f:
    creative_user_data = json.load(f)
print(len(creative_user_data)) 

def get_train_df(path):
    df = pl.scan_parquet(path)
    true = df.filter(pl.col('click_bool')==True)
    cnt = true.select(pl.len()).collect().to_numpy().flatten()[0]
    false = df.filter(pl.col('click_bool')==False).head(300*cnt)
    newdf = pl.concat([true,false]).collect(streaming=True).to_pandas()

    return newdf


def process_dataframe(df : pd.DataFrame) -> pd.DataFrame:
    df['adslotheight'] /= 100
    df['adslotwidth'] /= 100
    df['exchange'] = df['exchange'].astype('int8')
    df['day_of_week'] = df['day_of_week'].astype('str')
    print(df.columns)
    df['creative_user_count'] = df.apply(
        lambda row: sum(creative_user_data.get(str(row["creativeid"]), {}).get(uid, 0) 
        for uid in str(row["final_user_profile_id"]).split()), axis=1
    )
    print('user_hogye')
    tecols = ['region','city','device','browser','ad_type','day_of_week']
    te = ce.TargetEncoder()
    df[tecols] = te.fit_transform(df[tecols],df['click_bool'])

    print('te hogyi')
    
    scale_cols = ["region", "city", "device", "browser", "ad_type",'day_of_week']
    df[scale_cols] *= 100
    print('scale hogya')

    mms = MinMaxScaler((0,10))
    df[['creative_user_count','time_block']] = mms.fit_transform(df[['creative_user_count','time_block']])
    with open('encoders_scalers.pkl','wb') as f : pickle.dump({'te':te , 'mms':mms},f)
    return df

def get_xy(df):
    Xcols = ['region','city','exchange','adslotwidth','adslotheight','adslotvisibility','adslotformat','creative_user_count','time_block','day_of_week','device','browser','ad_type','advertiser_id']
    X = df[Xcols]
    bidY = df['bidding_price']
    clickY = df['click_bool']
    convY = df['conversion_bool']
    
    return X, bidY, clickY, convY

vx , bidy , ctry , cvry = get_xy(process_dataframe(pd.concat([get_train_df(path) for path in paths])))

74
Index(['bidId', 'region', 'city', 'exchange', 'adslotwidth', 'adslotheight',
       'adslotvisibility', 'adslotformat', 'adslotfloorprice', 'creativeid',
       'bidding_price', 'advertiser_id', 'impression_bool', 'click_bool',
       'conversion_bool', 'paying_price', 'final_user_profile_id', 'date',
       'time_block', 'day_of_week', 'device', 'browser', 'ad_type'],
      dtype='object')
user_hogye
te hogyi
scale hogya


In [69]:
def separate_advertisers (vx , ctry , cvry):
    advertisers = [1458,3358,3386,3427,3476]
    return [[vx[vx['advertiser_id']==k].drop('advertiser_id',axis=1), ctry[vx['advertiser_id']==k] , cvry[vx['advertiser_id']==k]] for k in advertisers]

separated = separate_advertisers(vx,ctry, cvry)
separated[0][0]


,region,city,exchange,adslotwidth,adslotheight,adslotvisibility,adslotformat,creative_user_count,time_block,day_of_week,device,browser,ad_type
0,0.346818,0.252813,2,3.36,2.8,2,0,0.000000,0.000000,0.106635,0.079242,0.080653,0.344067
1,0.412825,0.507419,3,3.00,2.5,0,0,0.000000,0.077720,0.106635,0.079242,0.080653,0.344067
2,0.306521,0.306521,3,9.60,0.9,0,0,0.000000,0.155440,0.106635,0.079242,0.080653,0.324992
3,0.380213,0.380213,1,3.36,2.8,2,1,0.000000,0.362694,0.106635,0.079242,0.080653,0.344067
4,0.253431,0.276932,3,10.00,0.9,0,0,0.949706,0.414508,0.106635,0.079242,0.080653,0.324992
...,...,...,...,...,...,...,...,...,...,...,...,...,...
898480,0.309456,0.357143,2,1.60,6.0,2,0,0.000000,0.077720,0.106635,0.886358,1.050739,0.307482
898481,0.303562,0.236587,2,7.28,0.9,1,0,0.000000,0.077720,0.106635,0.886358,1.050739,0.324992
898482,0.312210,0.363416,2,2.50,2.5,2,0,0.000000,0.077720,0.106635,0.886358,1.050739,0.344067
898483,0.306521,0.306521,2,3.36,2.8,2,0,0.000000,0.077720,0.106635,0.886358,1.000914,0.344067


In [70]:
def train_model(vx, ctry, name):
    X_train, X_valid, y_train, y_valid = train_test_split(vx, ctry, test_size=0.2, random_state=42)
    train_dataset = lgb.Dataset(X_train, label=y_train)
    valid_dataset = lgb.Dataset(X_valid, label=y_valid)

    params = {
        'objective': 'binary',
        'metric': ['auc', 'rmse'],
        'boosting_type': 'gbdt',
        'learning_rate': 0.1,
        'num_leaves': 31,
        'max_depth': -1,
        'verbose': -1
    }
    
    model = lgb.train(
        params=params,
        train_set=train_dataset,
        num_boost_round=1000,
        valid_sets=[train_dataset, valid_dataset],
        valid_names=['train', 'valid'],
        callbacks=[lgb.early_stopping(stopping_rounds=50)],
    )

    lgb_model_path = f"{name}.pkl"
    with open(lgb_model_path, "wb") as f:
        pickle.dump(model, f)

    print(f"LightGBM model saved to {lgb_model_path}")

In [71]:
advertisers = [1458,3358,3386,3427,3476]
for name,(x,cy,vy) in zip(advertisers, separated):
    train_model(x, cy, str(name))

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[10]	train's auc: 0.968665	train's rmse: 0.0238503	valid's auc: 0.947734	valid's rmse: 0.02517
LightGBM model saved to 1458.pkl
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[3]	train's auc: 0.916833	train's rmse: 0.0271794	valid's auc: 0.923022	valid's rmse: 0.0301901
LightGBM model saved to 3358.pkl
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[5]	train's auc: 0.939152	train's rmse: 0.027162	valid's auc: 0.932247	valid's rmse: 0.0258808
LightGBM model saved to 3386.pkl
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[10]	train's auc: 0.936464	train's rmse: 0.0272961	valid's auc: 0.928394	valid's rmse: 0.0271268
LightGBM model saved to 3427.pkl
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[9]	t

In [ ]:
cvrtraining = [separated[1] , separated[4]]


def train_cvr_model(vx, ctry, name):
    X_train, X_valid, y_train, y_valid = train_test_split(vx, ctry, test_size=0.2, random_state=42)
    train_dataset = lgb.Dataset(X_train, label=y_train)
    valid_dataset = lgb.Dataset(X_valid, label=y_valid)

    params = {
        'objective': 'binary',
        'metric': ['auc', 'rmse'],
        'boosting_type': 'gbdt',
        'learning_rate': 0.1,
        'num_leaves': 31,
        'max_depth': -1,
        'verbose': -1
    }

    model = lgb.train(
        params=params,
        train_set=train_dataset,
        num_boost_round=1000,
        valid_sets=[train_dataset, valid_dataset],
        valid_names=['train', 'valid'],
        callbacks=[lgb.early_stopping(stopping_rounds=100)],
    )

    lgb_model_path = f"{name}_cvr.pkl"
    with open(lgb_model_path, "wb") as f:
        pickle.dump(model, f)

    print(f"LightGBM model cvr saved to {lgb_model_path}")

for name,(x,cy,vy) in zip(['3358','3476'], cvrtraining):
    train_cvr_model(x,vy,name)


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1]	train's auc: 0.827247	train's rmse: 0.0390076	valid's auc: 0.813465	valid's rmse: 0.039542
LightGBM model cvr saved to 3358_cvr.pkl
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2]	train's auc: 0.18956	train's rmse: 0.0119167	valid's auc: 0.528684	valid's rmse: 0.0132932
LightGBM model cvr saved to 3476_cvr.pkl


In [84]:
def train_lgb_model(vx, bidy, model_name):
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(vx, bidy, test_size=0.2, random_state=42)
    
    # Define and train model
    model = lgb.LGBMRegressor(objective='regression', metric='l2', learning_rate=0.05, num_leaves=12, max_depth=-1, verbose=1,early_stopping_rounds=50)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='l2')
    
    # Predictions and RMSE evaluation
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"RMSE for {model_name}: {rmse}")
    
    # Save model
    model_path = f"{model_name}.pkl"
    with open(model_path, "wb") as f:
        pickle.dump(model, f)
    
    print(f"LightGBM model saved to {model_path}")
    return model

train_lgb_model(vx,bidy,'bid')

[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.114577 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 782
[LightGBM] [Info] Number of data points in the train set: 2600880, number of used features: 14
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Start training from score 269.698496
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.0349176
RMSE for bid: 0.18686262135735784
LightGBM model saved to bid.pkl


LGBMRegressor(early_stopping_rounds=50, learning_rate=0.05, metric='l2',
              num_leaves=12, objective='regression', verbose=1)